In [ ]:

from myhdl import block, delay, always_seq, instance, always, Signal, ResetSignal, traceSignals, now
from dataclasses import dataclass, field
from itertools import tee, product
from typing import Callable, Generator, Dict, List, Tuple, OrderedDict, Optional, Any, Union
from copy import deepcopy
from abc import ABC, abstractmethod, abstractproperty
import logging
from sys import version
import inspect
from functools import partial
# import showast
import ast
import astor
logging.basicConfig(level=logging.INFO)
logging.info(version)
traceSignals.filename = 'Top'
traceSignals.tracebackup = False
import islpy as isl


INFO:root:3.9.5 (default, Aug 29 2021, 19:01:31) 
[GCC 9.3.0]


In [192]:

# bset1 = isl.BasicSet("[n1, n2] -> {{[{}] : {} >= n1 and {} < n2}}".format('x', 'x', 'x'))
# bset2 = isl.BasicSet("{[x] : x >= 2 and x < 10}")
bset1 = isl.BasicSet("{[x] : 0<=x<10 }")
bmap1 = isl.BasicMap("[n] -> {[x] -> [x'] : x'=2*x+n}").intersect_params(isl.BasicSet("[n] -> { : 0<=n<=3}"))
# bmap2 = isl.BasicMap("{[x] -> [ACC] : ACC = x}").intersect_domain(bset1)
# print(bmap1)
# print(bmap2)
# print(bmap2.reverse())
# print(bmap1.apply_range(bmap2.reverse()))
# res = bset1.lex_lt_union_set(bset1)
# print(bset1.union(bset2))
# bset1.card()
print(bset1)
res = bmap1.intersect_domain(bset1)
print(res)


{ [x] : 0 <= x <= 9 }
[n] -> { [x] -> [x' = n + 2x] : 0 <= n <= 3 and 0 <= x <= 9 }


In [133]:
'a {{ {} }}'.format('x')

'a { x }'

In [134]:
# %%
@dataclass
class StreamStateControl:
    index_generator_fn: Generator
    initial_index_generator_fn: Generator = None
    _done: bool = False

    def __post_init__(self):
        self.index_generator_fn, self.initial_index_generator_fn = tee(
            self.index_generator_fn)

    def reset(self):
        self.done = False
        self.index_generator_fn = self.initial_index_generator_fn
        self.index_generator_fn, self.initial_index_generator_fn = tee(
            self.index_generator_fn)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            next_index = next(self.index_generator_fn)
        except StopIteration:
            next_index = 0
            self.done = True
        return next_index

    @property
    def done(self):
        return self._done

    @done.setter
    def done(self, val):
        if val:
            logging.debug("{} has concluded @T={}".format(self, now()))
        else:
            logging.debug("{} initialized @T={}".format(self, now()))
        self._done = val



@dataclass
class StreamTemplate:
    _generator_func_def: Generator
    _stream_start_time_default = 0
    _stream_defualt_value_default = 0
    _stream_array_accessed_default = None

    def _parameterized_stream_descriptor(self, gen_func_args, gen_func_kwargs, local_stream_start_time, local_stream_defualt_value):
        customized_generator = self._generator_func_def(*gen_func_args, **gen_func_kwargs)
        for _ in range(local_stream_start_time):
            yield local_stream_defualt_value
        yield from customized_generator

    def __call__(self, *args, **kwargs):
        if 'start_time' in kwargs:
            stream_start_time = kwargs['start_time']
            del kwargs['start_time']
        else:
            stream_start_time = self._stream_defualt_value_default
            
        if 'array_accessed' in kwargs:
            stream_array_accessed = kwargs['array_accessed']
            del kwargs['array_accessed']
        else:
            stream_array_accessed = self._stream_array_accessed_default
                        
        if 'default_val' in kwargs:
            stream_defualt_value = kwargs['default_val']
            del kwargs['default_val']
        else:
            stream_defualt_value = self._stream_defualt_value_default
            
        return self._parameterized_stream_descriptor(args, kwargs, stream_start_time, stream_defualt_value)


def stream(stream_def_func):
    new_stream = StreamTemplate(_generator_func_def=stream_def_func)
    return new_stream


In [135]:
# %%
@ block
def counter(clk, enable, reset, count):
    @always_seq(clk.posedge, reset=reset)
    def increment():
        if enable:
            count.next = count.val + 1
    return increment

@block
def clk_driver(clk, enable, period=20):
    lowTime = int(period / 2)
    highTime = period - lowTime

    @instance
    def drive_clk():
        while True:
            if not enable: 
                yield enable
            yield delay(lowTime)
            clk.next = 1
            yield delay(highTime)
            clk.next = 0

    return drive_clk

@ block
def stream_generator(clk, enable, reset, stream, stream_out):
    @always(clk.posedge, reset.posedge)
    def generate():
        if not reset and enable:
            if not stream.done:
                stream_out.next = next(stream)
        elif reset:
            stream.reset()
            stream_out.next = 0
    return generate


In [193]:
@stream
def chain_arch_pe_parameterizable_access_stream(c_ub, i_ub, j_ub, pe_channel, pe_group, pe, ifmap_dim):
    # Stream constants
    pe_start_index_offset = pe_channel*(ifmap_dim**2)+pe_group*ifmap_dim+pe
    # Dynamic computations
    for c in range(c_ub):
        for i in range(i_ub):
            for j in range(j_ub):
                yield i*ifmap_dim+j+pe_start_index_offset



# Layer Config
ifmap_dim = 10
kernel = 3
ofmap_dim = ifmap_dim-kernel+1
channel_count = 3

# Arch. Config For Full Channel Parallelism
pe_count = (kernel**2)*channel_count
pes_per_group = kernel
pes_per_channel = kernel**2
groups_per_channel = int(pes_per_channel/pes_per_group)
channel_chain_length = int(pe_count/pes_per_channel)


@block
def top():
    clk = Signal(bool(0))
    enable = Signal(bool(0))
    global_counter = Signal(0)
    reset = ResetSignal(bool(0), active=1, isasync=True)
    counter_inst = counter(clk, enable, reset, global_counter)
    clk_driver_inst = clk_driver(clk, enable, period=10)

    stream_out_list = [Signal(0) for _ in range(pe_count)]

    stream_generator_list = []
    for pe_channel in range(channel_chain_length):
        for pe_group in range(groups_per_channel):
            for pe in range(pes_per_group):
                pe_idx = pe_channel*pes_per_channel + pe_group*pes_per_group + pe
                stream_descriptor = chain_arch_pe_parameterizable_access_stream(
                    1, ofmap_dim, ofmap_dim, pe_channel, pe_group, pe, ifmap_dim, start_time=pe_idx)
                stream_state_controller = StreamStateControl(stream_descriptor)
                stream_generator_list.append(stream_generator(
                    clk, enable, reset, stream_state_controller, stream_out_list[pe_idx]))

    @instance
    def start_sim():
        # reset cycle
        enable.next = 0
        reset.next = 1
        yield delay(10)
        enable.next = 1
        reset.next = 0

    return clk_driver_inst, counter_inst, start_sim, stream_generator_list



In [137]:
# %%
dut = top()
inst = traceSignals(dut)
inst.run_sim(1200)
inst.quit_sim()


<class 'myhdl._SuspendSimulation'>: Simulated 1200 timesteps


In [138]:
utilization = ((pe_count+1)/2+((ofmap_dim**2)-(pe_count-1))+(pe_count+1)/2)/((pe_count-1)+ofmap_dim**2)
print(utilization)

0.7333333333333333


In [201]:
@stream
def example_func(c_ub, i_ub, j_ub, pe_channel, pe_group, pe, ifmap_dim, test = 1, *args, **kwargs):
    # Stream invariants
    pe_start_index_offset = pe_channel*(ifmap_dim**2)+pe_group*ifmap_dim+pe
    # Dynamic computations
    for c in range(c_ub, i_ub, j_ub):
        for i in range(i_ub):
            for j in range(j_ub):
                if not (1 == 1 and j_ub == 2 and asd==dsa and 4==ass):
                    if 4 == 3:
                        if 3 > a > 4:
                            yield i*ifmap_dim+j+pe_start_index_offset
                        else:
                            yield 3
                    else:
                        yield i*ifmap_dim+j+pe_start_index_offset
                        
                elif 3 == 3:
                    yield i*ifmap_dim+j+pe_start_index_offset
                else:
                    yield i*ifmap_dim+j+pe_start_index_offset

In [195]:
@dataclass
class IslIR:
    stream_name: str
    stream_args: ast.arguments
    yield_expr: List[Union[ast.Yield, Tuple[Union[ast.If, ast.Yield]]]] = field(default_factory=list)
    stream_invariant_assignments: List[ast.Assign] = field(
        default_factory=list)
    for_loops: List[ast.For] = field(default_factory=list)


class StreamTraverser(astor.ExplicitNodeVisitor):

    def visit_Module(self, node: ast.Module) -> Any:
        stream_func_def_node = node.body[0]
        self.visit_FunctionDef(stream_func_def_node)

    def visit_FunctionDef(self, node: ast.FunctionDef) -> Any:
        if node.decorator_list[0].id != 'stream':
            raise Exception("Invalid function used for conversion to ISL IR")
        self.isl_ir = IslIR(node.name, node.args)
        start_of_for_loops_idx = 0
        for idx, node in enumerate(node.body):
            if isinstance(node, ast.Assign):
                self.isl_ir.stream_invariant_assignments.append(node)
            elif isinstance(node, ast.For):
                start_of_for_loops_idx = idx
                break
            else:
                raise Exception(
                    "Invalid expression in function body, only constant assignments, and for loops are allowed")

        if start_of_for_loops_idx != len(node.body) - 1:
            logging.warning(
                "Other expressions in function definition beyond the first for loop will be ignored")
        print(node.body)
        self.visit_For(node.body[start_of_for_loops_idx])

    def visit_For(self, node: ast.For) -> Any:
        self.isl_ir.for_loops.append(node)
        first_entry_in_loop_body = node.body[0]
        if isinstance(first_entry_in_loop_body, ast.For):
            self.visit_For(first_entry_in_loop_body)
        elif isinstance(first_entry_in_loop_body, ast.If):
            self.visit_If(first_entry_in_loop_body)
        elif isinstance(first_entry_in_loop_body, ast.Expr):
            if isinstance(first_entry_in_loop_body.value, ast.Yield):
                self.visit_Yield(first_entry_in_loop_body.value)
            else:
                raise Exception("Invalid expression type in for loop body, only yield expressions are allowed")
        else:
            raise Exception(
                "Invalid for body, only for loops, if statements, and yield expressions are allowed in a for loop")

    def visit_Yield(self, node: ast.Yield) -> Any:
        self.isl_ir.yield_expr.append(node)

    def visit_If(self, node: ast.If, if_chain = ()) -> Any:
        if_chain += (node, )
        if_body = node.body + node.orelse
        if len(if_body) > 2:
            raise Exception("Too many expressions in if/elif/else statement bodies, something fishy is going on....")
        for entry in if_body:
            if isinstance(entry, ast.If):
                self.visit_If(entry, if_chain)
            elif isinstance(entry, ast.Expr):
                if(isinstance(entry.value, ast.Yield)):
                    self.isl_ir.yield_expr.append(if_chain)
                else:
                    raise Exception("Invalid expression type in if condition body, only yield expression are allowed")
            else:
                raise Exception("Invalid if condition body, only other if statements, and yield expressions are allowed")
        

# General flow
# Visit module
# Visit func def -> extract all arguments as dictionary, assign defaults from ast
# Visit constant assign statements, keep them as expressions you can evaluate later when all arguments are defined
# Keep Visit constant assign statements
# Get to to for loops,


In [202]:
tree = ast.parse(inspect.getsource(inspect.getgeneratorlocals(example_func(
                    1, ofmap_dim, ofmap_dim, 0, 0, 0, ifmap_dim, start_time=0))['self']._generator_func_def))


In [204]:
print(astor.dump(tree))

Module(
    body=[
        FunctionDef(name='example_func',
            args=arguments(posonlyargs=[],
                args=[arg(arg='c_ub', annotation=None, type_comment=None),
                    arg(arg='i_ub', annotation=None, type_comment=None),
                    arg(arg='j_ub', annotation=None, type_comment=None),
                    arg(arg='pe_channel', annotation=None, type_comment=None),
                    arg(arg='pe_group', annotation=None, type_comment=None),
                    arg(arg='pe', annotation=None, type_comment=None),
                    arg(arg='ifmap_dim', annotation=None, type_comment=None),
                    arg(arg='test', annotation=None, type_comment=None)],
                vararg=arg(arg='args', annotation=None, type_comment=None),
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=arg(arg='kwargs', annotation=None, type_comment=None),
                defaults=[Constant(value=1, kind=None)]),
            body=[
    

/tmp/ipykernel_27211/502768955.py:1: DeprecationWarning: astor.dump is deprecated.  Please use astor.dump_tree.
  print(astor.dump(tree))


In [178]:
StreamTraverser().visit(tree)

IndexError: list index out of range

In [198]:
type(range(10))

range

In [200]:
ast.parse(' \
if a > 3: \
    print(a) \
')

IndentationError: unexpected indent (<unknown>, line 1)

In [197]:
print(astor.dump(tree))

Module(
    body=[
        FunctionDef(name='example_func',
            args=arguments(posonlyargs=[],
                args=[arg(arg='c_ub', annotation=None, type_comment=None),
                    arg(arg='i_ub', annotation=None, type_comment=None),
                    arg(arg='j_ub', annotation=None, type_comment=None),
                    arg(arg='pe_channel', annotation=None, type_comment=None),
                    arg(arg='pe_group', annotation=None, type_comment=None),
                    arg(arg='pe', annotation=None, type_comment=None),
                    arg(arg='ifmap_dim', annotation=None, type_comment=None),
                    arg(arg='test', annotation=None, type_comment=None)],
                vararg=arg(arg='args', annotation=None, type_comment=None),
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=arg(arg='kwargs', annotation=None, type_comment=None),
                defaults=[Constant(value=1, kind=None)]),
            body=[
    

/tmp/ipykernel_27211/502768955.py:1: DeprecationWarning: astor.dump is deprecated.  Please use astor.dump_tree.
  print(astor.dump(tree))


In [183]:
list(range(10,20,1,4))

TypeError: range expected at most 3 arguments, got 4

In [184]:
inspect.getgeneratorlocals(example_func(
                    1, ofmap_dim, ofmap_dim, 0, 0, 0, ifmap_dim, 123, test = 2, start_time=0))

{'self': StreamTemplate(_generator_func_def=<function example_func at 0x7fe7dffcfca0>),
 'gen_func_args': (1, 8, 8, 0, 0, 0, 10, 123),
 'gen_func_kwargs': {'test': 2},
 'local_stream_start_time': 0,
 'local_stream_defualt_value': 0}